In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
teams_url = 'http://www.espn.com/nba/teams'
players_url = ''

In [3]:
html = urllib.request.urlopen(teams_url)
soup = BeautifulSoup(html, 'lxml')

In [4]:
links_store = str(soup.find_all('a'))

pattern = re.compile(r'href="/nba/team/_/name/\w*/\w*[-]?\w*')
matches = pattern.finditer(links_store)

teams = set()
for match in matches: 
    teams.add(match.group(0))

abbr_name = []
for link in teams:
    temp = link.split('/')
    abbr_name.append([temp[5], temp[6]])

abbr_name

[['cha', 'charlotte-hornets'],
 ['gs', 'golden-state'],
 ['chi', 'chicago-bulls'],
 ['ny', 'new-york'],
 ['det', 'detroit-pistons'],
 ['ind', 'indiana-pacers'],
 ['por', 'portland-trail'],
 ['den', 'denver-nuggets'],
 ['orl', 'orlando-magic'],
 ['dal', 'dallas-mavericks'],
 ['phx', 'phoenix-suns'],
 ['bkn', 'brooklyn-nets'],
 ['cle', 'cleveland-cavaliers'],
 ['mia', 'miami-heat'],
 ['sac', 'sacramento-kings'],
 ['min', 'minnesota-timberwolves'],
 ['tor', 'toronto-raptors'],
 ['mem', 'memphis-grizzlies'],
 ['bos', 'boston-celtics'],
 ['atl', 'atlanta-hawks'],
 ['phi', 'philadelphia-76ers'],
 ['sa', 'san-antonio'],
 ['utah', 'utah-jazz'],
 ['hou', 'houston-rockets'],
 ['wsh', 'washington-wizards'],
 ['okc', 'oklahoma-city'],
 ['mil', 'milwaukee-bucks'],
 ['no', 'new-orleans'],
 ['lac', 'la-clippers'],
 ['lal', 'los-angeles']]

In [5]:
roster_links = []
roster_link_format = 'https://www.espn.in/nba/team/roster/_/name/{abbr}'
for item in abbr_name:
    roster_links.append(roster_link_format.format(abbr=item[0]))
print(roster_links)

['https://www.espn.in/nba/team/roster/_/name/cha', 'https://www.espn.in/nba/team/roster/_/name/gs', 'https://www.espn.in/nba/team/roster/_/name/chi', 'https://www.espn.in/nba/team/roster/_/name/ny', 'https://www.espn.in/nba/team/roster/_/name/det', 'https://www.espn.in/nba/team/roster/_/name/ind', 'https://www.espn.in/nba/team/roster/_/name/por', 'https://www.espn.in/nba/team/roster/_/name/den', 'https://www.espn.in/nba/team/roster/_/name/orl', 'https://www.espn.in/nba/team/roster/_/name/dal', 'https://www.espn.in/nba/team/roster/_/name/phx', 'https://www.espn.in/nba/team/roster/_/name/bkn', 'https://www.espn.in/nba/team/roster/_/name/cle', 'https://www.espn.in/nba/team/roster/_/name/mia', 'https://www.espn.in/nba/team/roster/_/name/sac', 'https://www.espn.in/nba/team/roster/_/name/min', 'https://www.espn.in/nba/team/roster/_/name/tor', 'https://www.espn.in/nba/team/roster/_/name/mem', 'https://www.espn.in/nba/team/roster/_/name/bos', 'https://www.espn.in/nba/team/roster/_/name/atl', '

In [6]:
player_link_format = re.compile(r'https://www.espn.in/nba/player/_/id/\d*/\w*[-]?\w*')
player_list = set()
for roster_link in roster_links:
    roster_html = urllib.request.urlopen(roster_link)
    roster_soup = BeautifulSoup(roster_html, 'lxml')
    player_links = str(roster_soup.find_all('a'))
    players = player_link_format.finditer(player_links)
    for player in players:
        split_link = player.group(0).split('/')
        player_list.add((split_link[7], split_link[8]))
    time.sleep(2)
player_list = list(player_list)
player_list

[('6479', 'kemba-walker'),
 ('4066218', 'grant-williams'),
 ('2528693', 'torrey-craig'),
 ('3191', 'corey-brewer'),
 ('3992', 'james-harden'),
 ('3037789', 'bogdan-bogdanovic'),
 ('3155526', 'dillon-brooks'),
 ('2488653', 'mason-plumlee'),
 ('4423309', 'luka-samanic'),
 ('3907823', 'terance-mann'),
 ('6606', 'damian-lillard'),
 ('4066336', 'lauri-markkanen'),
 ('6426', 'davis-bertans'),
 ('3190', 'marco-belinelli'),
 ('4278508', 'troy-brown'),
 ('2530780', 'shabazz-napier'),
 ('984', 'tyson-chandler'),
 ('3423', 'goran-dragic'),
 ('4240', 'avery-bradley'),
 ('4395651', 'coby-white'),
 ('4230546', 'terrance-ferguson'),
 ('6430', 'jimmy-butler'),
 ('4395625', 'rj-barrett'),
 ('3059358', 'deonte-burton'),
 ('2581190', 'josh-richardson'),
 ('3138160', 'caleb-martin'),
 ('4401415', 'alen-smailagic'),
 ('3113587', 'cristiano-felicio'),
 ('6588', 'evan-fournier'),
 ('6477', 'jonas-valanciunas'),
 ('3133601', 'devonte-graham'),
 ('4278521', 'kz-okpala'),
 ('3135045', 'grayson-allen'),
 ('6589'

In [9]:
master_df = pd.DataFrame(columns = ['PNAME','YEAR','GP','GS','MIN','FG','FG%','3PT','3P%','FT','FT%','OR','DR','REB','AST','BLK','STL','PF','TO','PTS'])

player_stat_link_format = 'https://www.espn.in/nba/player/stats/_/id/{idno}/{name}'
player_name_stat_links = []
for name_id_pair in player_list:
    player_link = player_stat_link_format.format(idno=name_id_pair[0], name=name_id_pair[1])
    player_name_stat_links.append([name_id_pair[1], player_link])

for player_item in player_name_stat_links:
    print(player_item)
    player_link, pname = player_item[1], player_item[0]
    player_stat_html = urllib.request.urlopen(player_link)
    player_stat_soup = BeautifulSoup(player_stat_html, 'lxml')
    
    try:
        # fetching the year-team from the year table
        year_table = player_stat_soup.find_all('table', {'class':'Table Table--align-right Table--fixed Table--fixed-left'})[0]
        year_vals = year_table.find_all('tr')
        year_vals = [year_val.getText() for year_val in year_vals][1:-1]
        year_vals = [year[:4] for year in year_vals]

        # fetching the average season stats
        regular_season_averages = player_stat_soup.find_all('table', {'class':'Table Table--align-right'})[0]
        table_headers = regular_season_averages.find_all('th')
        table_rows = regular_season_averages.find_all('tr')
        all_rows = []
        for row in table_rows[:-1]:
            vals = row.find_all('td')
            vals = [val.getText() for val in vals]
            all_rows.append(vals)

        all_rows.remove([])   
        for idx, row in enumerate(all_rows):
            all_rows[idx] = [pname] + [year_vals[idx]] + row

        for i in range(len(all_rows)):
            idx = len(master_df)
            master_df.loc[idx] = all_rows[i]
    
        time.sleep(2)
    except Exception as e:
        pass


master_df
master_df.to_csv('nba_player_stats.csv', index=False)

['kemba-walker', 'https://www.espn.in/nba/player/stats/_/id/6479/kemba-walker']
['grant-williams', 'https://www.espn.in/nba/player/stats/_/id/4066218/grant-williams']
['torrey-craig', 'https://www.espn.in/nba/player/stats/_/id/2528693/torrey-craig']
['corey-brewer', 'https://www.espn.in/nba/player/stats/_/id/3191/corey-brewer']
['james-harden', 'https://www.espn.in/nba/player/stats/_/id/3992/james-harden']
['bogdan-bogdanovic', 'https://www.espn.in/nba/player/stats/_/id/3037789/bogdan-bogdanovic']
['dillon-brooks', 'https://www.espn.in/nba/player/stats/_/id/3155526/dillon-brooks']
['mason-plumlee', 'https://www.espn.in/nba/player/stats/_/id/2488653/mason-plumlee']
['luka-samanic', 'https://www.espn.in/nba/player/stats/_/id/4423309/luka-samanic']
['terance-mann', 'https://www.espn.in/nba/player/stats/_/id/3907823/terance-mann']
['damian-lillard', 'https://www.espn.in/nba/player/stats/_/id/6606/damian-lillard']
['lauri-markkanen', 'https://www.espn.in/nba/player/stats/_/id/4066336/lauri-

['deaaron-fox', 'https://www.espn.in/nba/player/stats/_/id/4066259/deaaron-fox']
['shaquille-harrison', 'https://www.espn.in/nba/player/stats/_/id/2991274/shaquille-harrison']
['tj-mcconnell', 'https://www.espn.in/nba/player/stats/_/id/2530530/tj-mcconnell']
['thanasis-antetokounmpo', 'https://www.espn.in/nba/player/stats/_/id/3102533/thanasis-antetokounmpo']
['trey-burke', 'https://www.espn.in/nba/player/stats/_/id/2579260/trey-burke']
['ante-zizic', 'https://www.espn.in/nba/player/stats/_/id/4017838/ante-zizic']
['trevor-ariza', 'https://www.espn.in/nba/player/stats/_/id/2426/trevor-ariza']
['tyler-herro', 'https://www.espn.in/nba/player/stats/_/id/4395725/tyler-herro']
['kristaps-porzingis', 'https://www.espn.in/nba/player/stats/_/id/3102531/kristaps-porzingis']
['tariq-owens', 'https://www.espn.in/nba/player/stats/_/id/3149059/tariq-owens']
['miye-oni', 'https://www.espn.in/nba/player/stats/_/id/4066851/miye-oni']
['joakim-noah', 'https://www.espn.in/nba/player/stats/_/id/3224/joak

['sindarius-thornwell', 'https://www.espn.in/nba/player/stats/_/id/3064539/sindarius-thornwell']
['donta-hall', 'https://www.espn.in/nba/player/stats/_/id/3907408/donta-hall']
['dwight-howard', 'https://www.espn.in/nba/player/stats/_/id/2384/dwight-howard']
['theo-pinson', 'https://www.espn.in/nba/player/stats/_/id/3138154/theo-pinson']
['jevon-carter', 'https://www.espn.in/nba/player/stats/_/id/3133635/jevon-carter']
['gorgui-dieng', 'https://www.espn.in/nba/player/stats/_/id/2534781/gorgui-dieng']
['frank-kaminsky', 'https://www.espn.in/nba/player/stats/_/id/2579294/frank-kaminsky']
['furkan-korkmaz', 'https://www.espn.in/nba/player/stats/_/id/3929325/furkan-korkmaz']
['jayson-tatum', 'https://www.espn.in/nba/player/stats/_/id/4065648/jayson-tatum']
['william-howard', 'https://www.espn.in/nba/player/stats/_/id/4237239/william-howard']
['rui-hachimura', 'https://www.espn.in/nba/player/stats/_/id/4066648/rui-hachimura']
['aron-baynes', 'https://www.espn.in/nba/player/stats/_/id/2968439

['ian-mahinmi', 'https://www.espn.in/nba/player/stats/_/id/2774/ian-mahinmi']
['georges-niang', 'https://www.espn.in/nba/player/stats/_/id/2990969/georges-niang']
['quinndary-weatherspoon', 'https://www.espn.in/nba/player/stats/_/id/3907358/quinndary-weatherspoon']
['nemanja-bjelica', 'https://www.espn.in/nba/player/stats/_/id/4269/nemanja-bjelica']
['lamarcus-aldridge', 'https://www.espn.in/nba/player/stats/_/id/2983/lamarcus-aldridge']
['jj-redick', 'https://www.espn.in/nba/player/stats/_/id/3024/jj-redick']
['willy-hernangomez', 'https://www.espn.in/nba/player/stats/_/id/2999409/willy-hernangomez']
['cory-joseph', 'https://www.espn.in/nba/player/stats/_/id/6446/cory-joseph']
['matt-thomas', 'https://www.espn.in/nba/player/stats/_/id/3059311/matt-thomas']
['patrick-beverley', 'https://www.espn.in/nba/player/stats/_/id/3964/patrick-beverley']
['cody-martin', 'https://www.espn.in/nba/player/stats/_/id/3138161/cody-martin']
['maxi-kleber', 'https://www.espn.in/nba/player/stats/_/id/2960

['malcolm-miller', 'https://www.espn.in/nba/player/stats/_/id/2580980/malcolm-miller']
['kendrick-nunn', 'https://www.espn.in/nba/player/stats/_/id/3056247/kendrick-nunn']
['damyean-dotson', 'https://www.espn.in/nba/player/stats/_/id/2991149/damyean-dotson']
['collin-sexton', 'https://www.espn.in/nba/player/stats/_/id/4277811/collin-sexton']
['trey-lyles', 'https://www.espn.in/nba/player/stats/_/id/3136196/trey-lyles']
['anfernee-simons', 'https://www.espn.in/nba/player/stats/_/id/4351851/anfernee-simons']
['jarrett-culver', 'https://www.espn.in/nba/player/stats/_/id/4277928/jarrett-culver']
['derrick-rose', 'https://www.espn.in/nba/player/stats/_/id/3456/derrick-rose']
['jeremiah-martin', 'https://www.espn.in/nba/player/stats/_/id/3907788/jeremiah-martin']
['anthony-tolliver', 'https://www.espn.in/nba/player/stats/_/id/3276/anthony-tolliver']
['bruce-brown', 'https://www.espn.in/nba/player/stats/_/id/4065670/bruce-brown']
['gary-harris', 'https://www.espn.in/nba/player/stats/_/id/2999

['jalen-brunson', 'https://www.espn.in/nba/player/stats/_/id/3934672/jalen-brunson']
['dante-exum', 'https://www.espn.in/nba/player/stats/_/id/3102528/dante-exum']
['tobias-harris', 'https://www.espn.in/nba/player/stats/_/id/6440/tobias-harris']
['james-johnson', 'https://www.espn.in/nba/player/stats/_/id/3999/james-johnson']
['serge-ibaka', 'https://www.espn.in/nba/player/stats/_/id/3439/serge-ibaka']
['kevon-looney', 'https://www.espn.in/nba/player/stats/_/id/3155535/kevon-looney']
['thaddeus-young', 'https://www.espn.in/nba/player/stats/_/id/3244/thaddeus-young']
['antonius-cleveland', 'https://www.espn.in/nba/player/stats/_/id/3064237/antonius-cleveland']
['stanley-johnson', 'https://www.espn.in/nba/player/stats/_/id/3134881/stanley-johnson']
['brian-bowen', 'https://www.espn.in/nba/player/stats/_/id/4277882/brian-bowen']
['elfrid-payton', 'https://www.espn.in/nba/player/stats/_/id/2583639/elfrid-payton']
['derrick-jones', 'https://www.espn.in/nba/player/stats/_/id/3936099/derrick-

In [13]:
df = pd.read_csv('nba_player_stats.csv')

In [15]:
df.head(10)

,PNAME,YEAR,GP,GS,MIN,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,kemba-walker,2011,66,25,27.2,4.3-11.6,36.6,1.0-3.4,30.5,2.5-3.2,78.9,0.5,3.1,3.5,4.4,0.3,0.9,1.2,1.8,12.1
1,kemba-walker,2012,82,82,34.9,6.4-15.2,42.3,1.3-4.0,32.2,3.6-4.5,79.8,0.7,2.7,3.5,5.7,0.4,2.0,1.9,2.4,17.7
2,kemba-walker,2013,73,73,35.8,6.2-15.7,39.3,1.5-4.5,33.3,3.9-4.6,83.7,0.5,3.7,4.2,6.1,0.4,1.2,1.8,2.3,17.7
3,kemba-walker,2014,62,58,34.2,6.1-15.8,38.5,1.4-4.5,30.4,3.8-4.6,82.7,0.6,3.0,3.5,5.1,0.5,1.4,1.5,1.6,17.3
4,kemba-walker,2015,81,81,35.6,7.0-16.4,42.7,2.2-6.0,37.1,4.6-5.4,84.7,0.7,3.7,4.4,5.2,0.5,1.6,1.4,2.1,20.9
5,kemba-walker,2016,79,79,34.7,8.1-18.3,44.4,3.0-7.6,39.9,3.8-4.5,84.7,0.6,3.3,3.9,5.5,0.3,1.1,1.5,2.1,23.2
6,kemba-walker,2017,80,80,34.2,7.4-17.0,43.1,2.9-7.5,38.4,4.5-5.3,86.4,0.4,2.7,3.1,5.6,0.3,1.1,1.2,2.2,22.1
7,kemba-walker,2018,82,82,34.9,8.9-20.5,43.4,3.2-8.9,35.6,4.6-5.5,84.4,0.6,3.8,4.4,5.9,0.4,1.2,1.6,2.6,25.6
8,kemba-walker,2019,56,56,31.1,6.8-15.9,42.5,3.2-8.4,38.1,3.7-4.3,86.4,0.6,3.2,3.9,4.8,0.5,0.9,1.6,2.1,20.4
9,grant-williams,2019,69,5,15.1,1.3-3.1,41.2,0.3-1.4,25.0,0.6-0.8,72.2,0.9,1.7,2.6,1.0,0.5,0.4,2.4,0.7,3.4
